In [1]:
%%writefile s_ga.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#define POP_SIZE 100  // Population size
#define NUM_TASKS 5   // Number of tasks
#define NUM_PROCESSORS 3  // Number of processors
#define MAX_GENERATIONS 500  // Maximum number of generations

// Function prototypes
void initializePopulation(int population[POP_SIZE][NUM_TASKS]);
int calculateCost(int assignment[NUM_TASKS]);
void selectParents(int population[POP_SIZE][NUM_TASKS], int parents[POP_SIZE][NUM_TASKS]);
void crossover(int parent1[NUM_TASKS], int parent2[NUM_TASKS], int child[NUM_TASKS]);
void mutate(int child[NUM_TASKS]);
void printAssignment(int assignment[NUM_TASKS]);

    int executionCost[NUM_TASKS][NUM_PROCESSORS] = {
        {10, 20, 30},
        {15, 50, 25},
        {30, 20, 10},
        {25, 15, 20},
        {20, 30, 15}
    };
    int communicationCost[NUM_TASKS][NUM_TASKS] = {
        {0, 5, 10, 15, 20},
        {5, 0, 5, 10, 15},
        {10, 5, 0, 5, 10},
        {15, 10, 5, 0, 5},
        {20, 15, 10, 5, 0}
    };

   int bestCost = 1000000;

int main() {
    srand(time(NULL));

    // Example task execution and communication costs


    int population[POP_SIZE][NUM_TASKS];  // Population of solutions
    int bestAssignment[NUM_TASKS];         // Best task assignment found
                // Initialize with a large number

    // Initialize population
    initializePopulation(population);

    // Genetic algorithm main loop
    for (int generation = 0; generation < MAX_GENERATIONS; generation++) {
        // Evaluate the cost of each assignment in the population
        for (int i = 0; i < POP_SIZE; i++) {
            int cost = calculateCost(population[i]);
            if (cost < bestCost) {
                bestCost = cost;
                for (int j = 0; j < NUM_TASKS; j++) {
                    bestAssignment[j] = population[i][j];
                }
            }
        }

        // Select parents for reproduction
        int parents[POP_SIZE][NUM_TASKS];
        selectParents(population, parents);

        // Generate new population through crossover and mutation
        for (int i = 0; i < POP_SIZE; i += 2) {
            int child1[NUM_TASKS], child2[NUM_TASKS];
            crossover(parents[i], parents[i + 1], child1);
            crossover(parents[i + 1], parents[i], child2);
            mutate(child1);
            mutate(child2);
            for (int j = 0; j < NUM_TASKS; j++) {
                population[i][j] = child1[j];
                population[i + 1][j] = child2[j];
            }
        }
    }

    printf("Best Assignment:\n");
    printAssignment(bestAssignment);
    return 0;
}

// Initialize the population with random task assignments
void initializePopulation(int population[POP_SIZE][NUM_TASKS]) {
    for (int i = 0; i < POP_SIZE; i++) {
        for (int j = 0; j < NUM_TASKS; j++) {
            population[i][j] = rand() % NUM_PROCESSORS;
        }
    }
}

// Calculate the total cost for a given task assignment
int calculateCost(int assignment[NUM_TASKS]) {
    // Example cost calculation, combining execution and communication costs
    int totalCost = 0;
    for (int i = 0; i < NUM_TASKS; i++) {
        totalCost += executionCost[i][assignment[i]];  // Execution cost
    }
    for (int i = 0; i < NUM_TASKS; i++) {
        for (int j = i + 1; j < NUM_TASKS; j++) {
            if (assignment[i] != assignment[j]) {
                totalCost += communicationCost[i][j];  // Communication cost
            }
        }
    }
    return totalCost;
}

// Select the best parents based on their fitness
void selectParents(int population[POP_SIZE][NUM_TASKS], int parents[POP_SIZE][NUM_TASKS]) {
    for (int i = 0; i < POP_SIZE; i++) {
        int parentIdx = rand() % POP_SIZE;
        for (int j = 0; j < NUM_TASKS; j++) {
            parents[i][j] = population[parentIdx][j];
        }
    }
}

// Perform crossover between two parent solutions
void crossover(int parent1[NUM_TASKS], int parent2[NUM_TASKS], int child[NUM_TASKS]) {
    int crossoverPoint = rand() % NUM_TASKS;
    for (int i = 0; i < crossoverPoint; i++) {
        child[i] = parent1[i];
    }
    for (int i = crossoverPoint; i < NUM_TASKS; i++) {
        child[i] = parent2[i];
    }
}

// Perform mutation on a child solution
void mutate(int child[NUM_TASKS]) {
    int mutationPoint = rand() % NUM_TASKS;
    child[mutationPoint] = rand() % NUM_PROCESSORS;
}

// Print the task assignment
void printAssignment(int assignment[NUM_TASKS]) {
    for (int i = 0; i < NUM_TASKS; i++) {
        printf("Task %d assigned to Processor %d\n", i + 1, assignment[i] + 1);
    }
    printf("\nBest Cost Found = %d",bestCost);
}

Writing s_ga.cu


In [3]:
!nvcc -arch=sm_75 s_ga.cu -o s_ga
!./s_ga

Best Assignment:
Task 1 assigned to Processor 3
Task 2 assigned to Processor 3
Task 3 assigned to Processor 3
Task 4 assigned to Processor 3
Task 5 assigned to Processor 3

Best Cost Found = 100

In [4]:
%%writefile p_ga.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define POP_SIZE 100
#define NUM_TASKS 5
#define NUM_PROCESSORS 3
#define MAX_GENERATIONS 500
#define MUTATION_RATE 0.1f

// Execution and communication costs
__device__ int executionCost[NUM_TASKS][NUM_PROCESSORS] = {
    {10, 20, 30},
    {15, 50, 25},
    {30, 20, 10},
    {25, 15, 20},
    {20, 30, 15}
};
__device__ int communicationCost[NUM_TASKS][NUM_TASKS] = {
    {0, 5, 10, 15, 20},
    {5, 0, 5, 10, 15},
    {10, 5, 0, 5, 10},
    {15, 10, 5, 0, 5},
    {20, 15, 10, 5, 0}
};

__global__ void initializePopulation(int *population, int seed) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP_SIZE * NUM_TASKS) {
        curandState state;
        curand_init(seed + idx, 0, 0, &state);
        population[idx] = curand(&state) % NUM_PROCESSORS;
    }
}

__device__ int calculateCost(int *assignment) {
    int cost = 0;
    for (int i = 0; i < NUM_TASKS; i++) {
        cost += executionCost[i][assignment[i]];
        for (int j = i + 1; j < NUM_TASKS; j++) {
            if (assignment[i] != assignment[j]) {
                cost += communicationCost[i][j];
            }
        }
    }
    return cost;
}

__global__ void evaluateFitness(int *population, int *fitness) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP_SIZE) {
        fitness[idx] = calculateCost(&population[idx * NUM_TASKS]);
    }
}

__global__ void crossoverAndMutate(int *population, int *new_population, int seed) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP_SIZE / 2) {
        curandState state;
        curand_init(seed + idx, 0, 0, &state);

        int parent1_idx = curand(&state) % POP_SIZE;
        int parent2_idx = curand(&state) % POP_SIZE;

        int crossoverPoint = curand(&state) % NUM_TASKS;
        for (int i = 0; i < crossoverPoint; i++) {
            new_population[idx * NUM_TASKS + i] = population[parent1_idx * NUM_TASKS + i];
            new_population[(idx + POP_SIZE / 2) * NUM_TASKS + i] = population[parent2_idx * NUM_TASKS + i];
        }
        for (int i = crossoverPoint; i < NUM_TASKS; i++) {
            new_population[idx * NUM_TASKS + i] = population[parent2_idx * NUM_TASKS + i];
            new_population[(idx + POP_SIZE / 2) * NUM_TASKS + i] = population[parent1_idx * NUM_TASKS + i];
        }

        // Mutation
        if (curand_uniform(&state) < MUTATION_RATE) {
            int mutationPoint = curand(&state) % NUM_TASKS;
            new_population[idx * NUM_TASKS + mutationPoint] = curand(&state) % NUM_PROCESSORS;
        }
    }
}

int main() {
    int *d_population, *d_new_population, *d_fitness;
    cudaMalloc(&d_population, POP_SIZE * NUM_TASKS * sizeof(int));
    cudaMalloc(&d_new_population, POP_SIZE * NUM_TASKS * sizeof(int));
    cudaMalloc(&d_fitness, POP_SIZE * sizeof(int));

    initializePopulation<<<(POP_SIZE * NUM_TASKS + 255) / 256, 256>>>(d_population, time(NULL));

    for (int gen = 0; gen < MAX_GENERATIONS; gen++) {
        evaluateFitness<<<(POP_SIZE + 255) / 256, 256>>>(d_population, d_fitness);
        crossoverAndMutate<<<(POP_SIZE / 2 + 255) / 256, 256>>>(d_population, d_new_population, time(NULL));
        cudaMemcpy(d_population, d_new_population, POP_SIZE * NUM_TASKS * sizeof(int), cudaMemcpyDeviceToDevice);
    }

    int h_population[POP_SIZE * NUM_TASKS];
    int h_fitness[POP_SIZE];
    cudaMemcpy(h_population, d_population, POP_SIZE * NUM_TASKS * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_fitness, d_fitness, POP_SIZE * sizeof(int), cudaMemcpyDeviceToHost);

    int bestIdx = 0;
    for (int i = 1; i < POP_SIZE; i++) {
        if (h_fitness[i] < h_fitness[bestIdx]) {
            bestIdx = i;
        }
    }

    printf("Best Assignment:\n");
    for (int i = 0; i < NUM_TASKS; i++) {
        printf("Task %d assigned to Processor %d\n", i + 1, h_population[bestIdx * NUM_TASKS + i] + 1);
    }
    printf("\nBest Cost Found = %d\n", h_fitness[bestIdx]);

    cudaFree(d_population);
    cudaFree(d_new_population);
    cudaFree(d_fitness);
    return 0;
}

Writing p_ga.cu
